# Lab 1: LoRA & QLoRA - Fine-Tuning a Llama-2 Model
---
## Notebook 2: The Training Process

**Goal:** In this notebook, you will use QLoRA to fine-tune a `Llama-2-7B` model on an instruction-following dataset.

**You will learn to:**
-   Load a dataset for supervised fine-tuning and preprocess it with a tokenizer.
-   Configure `BitsAndBytesConfig` to load a model in 4-bit precision (QLoRA).
-   Load a pre-trained Llama-2 model for causal language modeling.
-   Configure `peft.LoraConfig` to apply LoRA adapters to the model.
-   Use the `transformers.Trainer` to efficiently fine-tune the model.

### Step 1: Load Dataset and Preprocess

First, we'll load our instruction-tuning dataset. We will use the `guanaco-llama2-1k` dataset, which is a small, high-quality dataset of prompts and responses formatted for Llama-2.

#### Key Hugging Face Components:

-   `datasets.load_dataset`: Fetches a dataset from the Hugging Face Hub.
-   `transformers.AutoTokenizer`: Loads the appropriate tokenizer for our model.
-   `dataset.map()`: A powerful method to apply a processing function to every example in the dataset. We use `batched=True` for efficient processing.

### **`mlabonne/guanaco-llama2-1k` 資料集**

#### **為什麼選擇它？**
- **格式相容**：專為 Llama 2 提示格式設計，無需額外處理。  
- **輕量高效**：1000 條樣本，適合快速微調與測試。  
- **開發者友好**：附官方教學與 Colab Notebook，易於上手。

#### **特點**
- **高質量來源**：基於 OpenAssistant 數據，涵蓋多樣化指令與回答場景。  
- **即插即用**：格式化完成，支持 Llama 2 微調需求。

#### **缺點**
- **數據量小**：僅 1000 條樣本，適合小型實驗但不適用於大規模訓練。  
- **覆蓋範圍有限**：樣本多樣性不足，需額外擴展應用場景。

#### **展望**
- **數據擴展**：結合其他資料集，擴充樣本規模。  
- **遷移學習**：作為初始微調基礎，進一步提升性能。  
- **定制化優化**：針對特定領域進行專屬微調。

#### **結論**
`mlabonne/guanaco-llama2-1k` 是一款輕量高效的資料集，為 Llama 2 微調提供理想基礎，適合快速實驗與模型測試。

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "NousResearch/Llama-2-7b-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Important for Causal LM

# --- Load Dataset ---
dataset = load_dataset(dataset_name, split="train")

# --- Preprocess and Tokenize ---
def preprocess_function(examples):
    # The 'text' field contains the full formatted prompt.
    return tokenizer(examples["text"], truncation=True, max_length=1024)

# Split the dataset
dataset = dataset.train_test_split(test_size=0.1)

# Tokenize both splits
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"]) # Remove original text column

print("✅ Dataset loaded and tokenized.")
print(f"Train samples: {len(tokenized_datasets['train'])}")
print(f"Test samples: {len(tokenized_datasets['test'])}")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Dataset loaded and tokenized.
Train samples: 900
Test samples: 100


### Step 2: Load the Base Model

Now, we load the base model. The key to QLoRA is loading the base model in a quantized format.

#### Key Hugging Face `transformers` Components:

-   `transformers.BitsAndBytesConfig`: This configuration class is used to specify all the parameters for quantization.
    -   `load_in_4bit=True`: This is the master switch to enable 4-bit loading.
    -   `bnb_4bit_quant_type="nf4"`: We use the "nf4" (Normalized Float 4) quantization type, which is recommended for QLoRA.
    -   `bnb_4bit_compute_dtype=torch.bfloat16`: This sets the compute data type during the forward and backward passes. `bfloat16` is a good choice for modern GPUs.
-   `transformers.AutoModelForCausalLM`: We use this to load our Llama-2 model, passing the `quantization_config` to it.


好的，這是在 `02-Train.ipynb` 檔案中對 `Normalized Float 4 (NF4)` 核心思想的精簡 Markdown 表述，您可以將其整合到您的筆記中。

---

### `Normalized Float 4 (NF4)` 核心思想解析

`Normalized Float 4 (NF4)` 是 QLoRA 中使用的一種創新的 4-bit 資料型態，其核心理念是：**為常態分佈的模型權重，設計一種資訊理論上最優的 4-bit 量化方法**。

#### 核心方法：分位數量化 (Quantile Quantization)

1.  **問題前提**：
    大型語言模型的權重（weights）並非均勻分佈，而是高度集中於 0 附近，呈現**常態分佈**（鐘形曲線）。傳統的 4-bit 格式（如 FP4）是均勻量化的，這會導致在權重密集區精度損失較大。

2.  **NF4 的解決方案**：
    *   **建立理想模板**：首先，演算法會建立一個標準常態分佈的「模板」。
    *   **計算分位數**：接著，它將這個模板劃分為 `2^4 = 16` 個**等機率**的區間。這意味著每個區間包含的數據點數量是相同的。
    *   **非均勻量化**：這 16 個區間的分割點就成為了 NF4 的量化值。結果是一種非均勻的刻度，在數據密集的中心區域（靠近 0）刻度更精細，而在數據稀疏的兩端區域刻度較粗。

3.  **類比**：
    *   傳統 FP4 像一把**普通尺**，刻度是均勻的。
    *   NF4 則像一把**特製的對數尺**，專為測量常態分佈而設計，在最需要的地方提供最高精度。

#### 主要優勢

*   **更少資訊損失**：相比傳統 4-bit 格式，NF4 在量化過程中能保留更多原始權重的有效資訊。
*   **更高模型性能**：由於量化誤差更小，使用 QLoRA 微調後的模型表現更接近於使用 bfloat16 等更高精度的結果。

https://lh3.googleusercontent.com/d/1-Ms-lrWOQkFsPG_D9oMcA1UrWzVKSdC7
https://i.ytimg.com/vi/aZPAqBov3tQ/maxresdefault.jpg


In [2]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# --- Quantization Configuration ---
# Load model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# --- Load Base Model ---
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

print("✅ Base model loaded successfully!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Base model loaded successfully!


### Step 3: Configure LoRA

Now we configure LoRA using the `peft` library.

#### Key Hugging Face `peft` Components:

-   `peft.LoraConfig`: The main configuration class for LoRA.
    -   `r`: The rank of the update matrices. A lower rank means fewer trainable parameters. A common range is 8-64.
    -   `lora_alpha`: The scaling factor for the LoRA matrices. It's often set to twice the rank (`2*r`).
    -   `target_modules`: A list of the names of the modules (e.g., attention layers) to apply LoRA to. For Llama models, this is typically `["q_proj", "k_proj", "v_proj", "o_proj"]`.
    -   `lora_dropout`: Dropout probability for the LoRA layers to reduce overfitting.
    -   `bias="none"`: Specifies which biases to train. "none" is common.
    -   `task_type="CAUSAL_LM"`: Specifies the task type.
-   `peft.get_peft_model`: This function takes the base model and the LoRA config and returns a PEFT model ready for training.

In [3]:
from peft import LoraConfig, get_peft_model, TaskType

# --- LoRA Configuration ---
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# --- Create PEFT Model ---
peft_model = get_peft_model(model, lora_config)

# --- Print Trainable Parameters ---
peft_model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


### Step 4: Set Up Training

The final step is to configure and run the training process using the `transformers.Trainer`.

#### Key Hugging Face Components:

-   `transformers.TrainingArguments`: This class holds all the hyperparameters for the training run, such as learning rate, number of epochs, batch size, and logging settings.
    -   `metric_for_best_model="perplexity"`: We specify perplexity as the metric to determine the best model.
    -   `greater_is_better=False`: Lower perplexity indicates better performance.
-   `transformers.Trainer`: The standard trainer class from the `transformers` library. It requires a model, training arguments, datasets, a tokenizer, and a data collator.
-   `transformers.DataCollatorForLanguageModeling`: This data collator will be used to form batches of tokenized data. It also handles the creation of the `labels` for causal language modeling, where the model predicts the next token.
-   `compute_metrics`: A custom function to calculate **perplexity**, which is the standard evaluation metric for language modeling tasks. Perplexity measures how well the model predicts the next token - lower values indicate better performance.

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback
import torch
import math
import numpy as np

# --- Custom Evaluation Metrics ---
def compute_metrics(eval_pred):
    """
    Compute perplexity from the model's predictions.
    Perplexity is the standard metric for language modeling tasks.
    """
    predictions, labels = eval_pred
    
    # Convert to tensors
    predictions = torch.from_numpy(predictions).float()
    labels = torch.from_numpy(labels).long()
    
    # Shift for causal language modeling
    shift_logits = predictions[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    # Calculate cross entropy loss
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)), 
        shift_labels.view(-1)
    )

    # Calculate perplexity
    try:
        perplexity = math.exp(loss.item())
    except OverflowError:
        perplexity = float('inf')
    
    return {
        "perplexity": perplexity,
        "eval_loss": loss.item()
    }

# --- Data Collator ---
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- Gradient & Model Config ---
peft_model.train()
peft_model.enable_input_require_grads()      # ✅ 確保梯度可回傳
peft_model.gradient_checkpointing_enable()   # ✅ 節省顯存
peft_model.config.use_cache = False          # ⚠️ 需關閉 cache 以啟用反傳

# Freeze all except LoRA layers
for name, param in peft_model.named_parameters():
    if 'lora_' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in peft_model.parameters())
print(f"✅ Gradient configuration complete! Trainable: {trainable_params:,} / Total: {total_params:,}")

# 🧹 清理GPU記憶體
torch.cuda.empty_cache()
print("🧹 GPU memory cleared before training")

# --- Production Training Arguments for RTX 2000 Ada (15.6 GB) ---
# ✅ STABLE CONFIG: 已確認記憶體穩定，升級至正式訓練配置 + Early Stopping
training_args = TrainingArguments(
    output_dir="./lora-llama2-7b-guanaco",
    overwrite_output_dir=True,
    
    # 📈 學習策略
    learning_rate=3e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,                       # 3% warmup for stable learning
    weight_decay=0.01,                       # L2 regularization
    
    # 🎯 訓練設定 - 平衡效能與穩定性
    per_device_train_batch_size=1,           # 保持最小以確保記憶體穩定
    per_device_eval_batch_size=1,            # 評估批次也保持最小
    gradient_accumulation_steps=32,          # 恢復原值模擬 batch_size=16
    max_grad_norm=1.0,                       # 梯度裁剪防止爆炸
    
    # 🕐 訓練週期
    num_train_epochs=3,                      # 增加 epochs 讓 early stopping 有機會生效
    max_steps=1000,                          # 增加最大步數上限
    
    # 💾 記憶體最佳化 (保持穩定配置)
    fp16=True,                              # 半精度訓練
    gradient_checkpointing=True,            # 梯度檢查點節省記憶體
    optim="paged_adamw_8bit",              # 8-bit 優化器
    dataloader_pin_memory=False,           # 關閉以避免RAM壓力
    dataloader_num_workers=0,              # 單程序避免記憶體競爭
    remove_unused_columns=True,            # 移除未使用資料欄位
    
    # 📊 評估與儲存策略
    eval_strategy="steps",                  # 定期評估
    eval_steps=50,                         # 縮短評估間隔以便 early stopping 更敏感
    save_strategy="steps",
    save_steps=50,                         # 與評估同步保存
    save_total_limit=3,                    # 保留最近3個檢查點
    load_best_model_at_end=True,           # 載入最佳模型
    
    # 📏 評估指標 & Early Stopping
    metric_for_best_model="eval_perplexity", # 以困惑度為最佳模型指標
    greater_is_better=False,               # 越低越好 (perplexity)
    
    # 📝 日誌設定
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,                      # 每10步記錄一次
    logging_first_step=True,
    
    # 🔕 報告設定
    report_to=None,                        # 不上傳至外部平台
    disable_tqdm=False,                    # 保留進度條
    
    # 🚀 進階設定
    push_to_hub=False,                     # 不推送至 Hub
    hub_token=None,
    ignore_data_skip=True,                 # 忽略資料跳過警告
)

# --- Early Stopping Callback Configuration ---
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,              # 連續5次評估無改善就停止
    early_stopping_threshold=0.01,          # 改善閾值 (perplexity 下降 < 0.01 視為無改善)
)

# --- Create Trainer with Early Stopping ---
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],     # 恢復評估資料集
    data_collator=data_collator,
    compute_metrics=compute_metrics,             # 恢復指標計算
    callbacks=[early_stopping_callback],        # ✅ 加入 Early Stopping
)

# --- Start Production Training with Early Stopping ---
print("🚀 Starting PRODUCTION QLoRA training with Early Stopping...")
print(f"📊 Training config: {training_args.per_device_train_batch_size} batch × {training_args.gradient_accumulation_steps} accum = {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps} effective batch")
print(f"🎯 Max epochs: {training_args.num_train_epochs}, Max steps: {training_args.max_steps}")
print(f"📈 Evaluation every {training_args.eval_steps} steps")
print(f"⏹️  Early stopping: patience={early_stopping_callback.early_stopping_patience}, threshold={early_stopping_callback.early_stopping_threshold}")
print(f"💾 Memory-optimized: fp16={training_args.fp16}, gradient_checkpointing={training_args.gradient_checkpointing}")

# Train with automatic early stopping
train_result = trainer.train()
print("✅ Production training complete!")

# Show training summary
if hasattr(train_result, 'metrics'):
    print(f"\n📈 Training Summary:")
    print(f"Total steps completed: {train_result.metrics.get('train_steps', 'N/A')}")
    print(f"Training runtime: {train_result.metrics.get('train_runtime', 'N/A'):.2f}s" if 'train_runtime' in train_result.metrics else "")
    
    # Check if training was stopped early
    if train_result.metrics.get('train_steps', 0) < training_args.max_steps:
        print("⏹️  Training stopped early due to no improvement!")
    else:
        print("🏁 Training completed full course!")

# --- Final Evaluation ---
print("\n📊 Final Evaluation Results:")
final_metrics = trainer.evaluate()
print(f"Final Perplexity: {final_metrics['eval_perplexity']:.4f}")
print(f"Final Loss: {final_metrics['eval_loss']:.4f}")
print(f"Training Samples: {len(tokenized_datasets['train'])}")
print(f"Evaluation Samples: {len(tokenized_datasets['test'])}")

# Display early stopping information
print(f"\n⏹️  Early Stopping Configuration:")
print(f"   - Patience: {early_stopping_callback.early_stopping_patience} evaluations")
print(f"   - Threshold: {early_stopping_callback.early_stopping_threshold} perplexity improvement")
print(f"   - Evaluation frequency: Every {training_args.eval_steps} steps")

✅ Gradient configuration complete! Trainable: 4,194,304 / Total: 3,504,607,232
🧹 GPU memory cleared before training
[2025-10-15 12:27:24,422] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.


/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


🚀 Starting PRODUCTION QLoRA training with Early Stopping...
📊 Training config: 1 batch × 32 accum = 32 effective batch
🎯 Max epochs: 3, Max steps: 1000
📈 Evaluation every 50 steps
⏹️  Early stopping: patience=3, threshold=0.01
💾 Memory-optimized: fp16=True, gradient_checkpointing=True


Step,Training Loss,Validation Loss
